In [1]:
#import the neccesary libraries
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
from PIL import Image
import numpy as np
import skimage
import skimage.transform
import os
import tensorflow.keras.backend as K
from skimage.metrics import structural_similarity as ssim
import math
from tensorflow.keras.applications.vgg16 import VGG16
from sobel_edge import sobel 

os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [2]:
import skimage.transform

def CreateSet(input_list,input_dir,output_dir,N_patches,patchsize,H,W):
    """
    Creates an array that can be used as input and output for the model
    parameters: input_list, a list with indices of the images that are used
                N_patches, the amount of image patches the model creates per image
                patchsize, the width and height of the square image patch
    
    """
    X = np.zeros((len(input_list)*N_patches, patchsize, patchsize, 1), dtype=np.float32)
    y = np.zeros((len(input_list)*N_patches, patchsize, patchsize, 3), dtype=np.float32)
    for j in range(N_patches):
        for n,i in enumerate(input_list):
            #random pixel location
            ii = j*len(input_list)+n
            xloc = random.randint(0,W-patchsize)
            yloc = random.randint(0,H-patchsize)
            #input 
            img_input = Image.open(input_dir+'/input_%03d.png' % (i+1))
            in_img_input = tf.keras.preprocessing.image.img_to_array(img_input)
            in_img_input = in_img_input[yloc:yloc+patchsize,xloc:xloc+patchsize]
            in_img_input = skimage.transform.resize(in_img_input , (patchsize , patchsize , 1) , mode = 'constant' , preserve_range = True)
            X[ii] = in_img_input / 255.0

            #output
            img_output = Image.open(output_dir+'/target_%03d.png' % (i+1))
            in_img_output = tf.keras.preprocessing.image.img_to_array(img_output)
            in_img_output = in_img_output[yloc:yloc+patchsize,xloc:xloc+patchsize]
            in_img_output = skimage.transform.resize(in_img_output , (patchsize , patchsize , 1) , mode = 'constant' , preserve_range = True)
            in_img_output = np.concatenate((in_img_output,in_img_output,in_img_output),axis=-1)
            y[ii] = in_img_output / 255.0
            if ii % 1000 == 0:
                print(ii)
    print('Done')
    return X,y
#This function can be used for small datasets 
#amount of patches per image
N_patches = 64
#width and height of the patches
patchsize = 128
#image dimensions
H=801
W=401
input_dir = 'bandLimited'
output_dir = 'groundTruth'
# input_dir = 'CIRSBandLimited'
# output_dir = 'CIRSGroundTruth'
# input_dir = 'carotidBandLimited'
# output_dir = 'carotidGroundTruth'

#Train set is image 1 till 401
X_train,y_train = CreateSet(range(401),input_dir,output_dir,N_patches,patchsize,H,W)
#validation set is image 402 till 535
X_valid,y_valid = CreateSet(range(401,535),input_dir,output_dir,N_patches,patchsize,H,W)

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
Done
0
1000
2000
3000
4000
5000
6000
7000
8000
Done


In [2]:
def SRCNN():
    # lrelu = LeakyReLU(alpha=0.1)
    SRCNN = tf.keras.Sequential()
    SRCNN.add(tf.keras.layers.Conv2D(filters=128, kernel_size= (9,9),  kernel_initializer='glorot_uniform',
                     activation='relu', padding='same', input_shape=(None, None, 1)))
    SRCNN.add(tf.keras.layers.Conv2D(filters=64, kernel_size= (3,3),  kernel_initializer='glorot_uniform',
                     activation='relu', padding='same'))
    # SRCNN.add(BatchNormalization())
    SRCNN.add(tf.keras.layers.Conv2D(filters=3, kernel_size= (5,5),  kernel_initializer='glorot_uniform',
                     activation='linear', padding='same'))
    adam = tf.keras.optimizers.Adam(lr=0.0003)
    SRCNN.compile(optimizer=adam, loss=loss, loss_weights =loss_weights, metrics=[tf.keras.metrics.RootMeanSquaredError()])
    return SRCNN


In [4]:
image_shape = (None,None, 3)
vgg = VGG16(include_top=False, weights='imagenet', input_shape=image_shape)
selectedLayers = ['block1_conv2','block2_conv2','block3_conv3','block4_conv3']
selectedOutputs = [vgg.get_layer(i).output for i in selectedLayers]
loss_model = tf.keras.Model(inputs=vgg.input, outputs=selectedOutputs)
loss_model.trainable = False
def perceptual_loss(y_true, y_pred):
    mse = 0
    for i in range(0,3):
        mse = (mse+ K.mean(K.square(loss_model(y_true)[i] - loss_model(y_pred)[i])))
    return mse

loss = [perceptual_loss, tf.keras.losses.MeanSquaredError]
loss_weights = [70,30]

In [5]:
#define and compile the model
model = SRCNN()

In [6]:
model_path = 'model_SRCNN_Perceptual_ultrasound.h5'
callbacks = [
    tf.keras.callbacks.EarlyStopping(patience=10, verbose=1),
    tf.keras.callbacks.ReduceLROnPlateau(factor=0.1, patience=5, min_lr=0.0001, verbose=1),
    tf.keras.callbacks.ModelCheckpoint(model_path, verbose=1, save_best_only=True, save_weights_only=True)
]

In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, None, None, 128)   31232     
_________________________________________________________________
conv2d_1 (Conv2D)            (None, None, None, 64)    73792     
_________________________________________________________________
conv2d_2 (Conv2D)            (None, None, None, 3)     4803      
Total params: 109,827
Trainable params: 109,827
Non-trainable params: 0
_________________________________________________________________


In [15]:
#train the model
results = model.fit(X_train,y_train, epochs=100, callbacks=callbacks,batch_size = 16,
                    validation_data=(X_valid,y_valid),shuffle=False)


Epoch 1/100
1604/1604 [==============================] - ETA: 0s - loss: 2209.3740 - root_mean_squared_error: 0.1385
Epoch 00001: val_loss improved from inf to 2113.26196, saving model to model_SRCNN_Perceptual_ultrasound.h5
1604/1604 [==============================] - 130s 81ms/step - loss: 2209.3740 - root_mean_squared_error: 0.1385 - val_loss: 2113.2620 - val_root_mean_squared_error: 0.1184 - lr: 3.0000e-04
Epoch 2/100
1604/1604 [==============================] - ETA: 0s - loss: 2055.0076 - root_mean_squared_error: 0.1180
Epoch 00002: val_loss improved from 2113.26196 to 2061.20410, saving model to model_SRCNN_Perceptual_ultrasound.h5
1604/1604 [==============================] - 131s 81ms/step - loss: 2055.0076 - root_mean_squared_error: 0.1180 - val_loss: 2061.2041 - val_root_mean_squared_error: 0.1164 - lr: 3.0000e-04
Epoch 3/100
1604/1604 [==============================] - ETA: 0s - loss: 2028.3986 - root_mean_squared_error: 0.1176
Epoch 00003: val_loss improved from 2061.20410 t

Epoch 21/100
1604/1604 [==============================] - ETA: 0s - loss: 1961.9257 - root_mean_squared_error: 0.1184
Epoch 00021: val_loss improved from 1983.62659 to 1982.56067, saving model to model_SRCNN_Perceptual_ultrasound.h5
1604/1604 [==============================] - 129s 80ms/step - loss: 1961.9257 - root_mean_squared_error: 0.1184 - val_loss: 1982.5607 - val_root_mean_squared_error: 0.1186 - lr: 3.0000e-04
Epoch 22/100
1604/1604 [==============================] - ETA: 0s - loss: 1960.8684 - root_mean_squared_error: 0.1186
Epoch 00022: val_loss improved from 1982.56067 to 1982.03223, saving model to model_SRCNN_Perceptual_ultrasound.h5
1604/1604 [==============================] - 129s 80ms/step - loss: 1960.8684 - root_mean_squared_error: 0.1186 - val_loss: 1982.0322 - val_root_mean_squared_error: 0.1189 - lr: 3.0000e-04
Epoch 23/100
1604/1604 [==============================] - ETA: 0s - loss: 1959.9551 - root_mean_squared_error: 0.1189
Epoch 00023: val_loss improved from 19

Epoch 41/100
1604/1604 [==============================] - ETA: 0s - loss: 1947.6954 - root_mean_squared_error: 0.1229
Epoch 00041: val_loss improved from 1974.12341 to 1973.30811, saving model to model_SRCNN_Perceptual_ultrasound.h5
1604/1604 [==============================] - 129s 81ms/step - loss: 1947.6954 - root_mean_squared_error: 0.1229 - val_loss: 1973.3081 - val_root_mean_squared_error: 0.1220 - lr: 3.0000e-04
Epoch 42/100
1604/1604 [==============================] - ETA: 0s - loss: 1947.2200 - root_mean_squared_error: 0.1230
Epoch 00042: val_loss did not improve from 1973.30811
1604/1604 [==============================] - 129s 80ms/step - loss: 1947.2200 - root_mean_squared_error: 0.1230 - val_loss: 1974.2150 - val_root_mean_squared_error: 0.1221 - lr: 3.0000e-04
Epoch 43/100
1604/1604 [==============================] - ETA: 0s - loss: 1946.7924 - root_mean_squared_error: 0.1232
Epoch 00043: val_loss did not improve from 1973.30811
1604/1604 [==============================] - 

KeyboardInterrupt: 

In [7]:
#load the model
model.load_weights(model_path)

In [ ]:
#visualise the accuracy and losses of the model
# list all data in history
print(results.history.keys())
# summarize history for accuracy
plt.plot(results.history['root_mean_squared_error'])
plt.plot(results.history['val_root_mean_squared_error'])
plt.title('model root_mean_squared_error')
plt.ylabel('root_mean_squared_error')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(results.history['loss'])
plt.plot(results.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [16]:
Train_set_indices = range(535)
Train_set_X = np.zeros((len(Train_set_indices), 801, 401,1), dtype=np.float32)
for i in Train_set_indices:
    img = Image.open('bandLimited/input_%03d.png' % (i+1))
    in_img_input = tf.keras.preprocessing.image.img_to_array(img)
    in_img_input = skimage.transform.resize(in_img_input , (801 , 401 , 1) , mode = 'constant' , preserve_range = True)
    Train_set_X[i] =in_img_input / 255.0

In [18]:
prediction_images = np.zeros((len(Train_set_X),801,401),dtype=np.float32)
for i in Train_set_indices:
    prediction_image = model.predict(np.expand_dims(Train_set_X[i] , 0))
    prediction_image = prediction_image[:,:,:,0]
    prediction_image = prediction_image.squeeze()*255.0
    prediction_image  = Image.fromarray(prediction_image)
    if prediction_image.mode == "F":
        prediction_image = prediction_image.convert('RGB')
    prediction_image.save('bandlimited_SRCNNmodel1/input_%03d.png' % (i+1))


In [8]:
def CreateTestSet(test_set_indices,input_dir_test,output_dir_test):
    """
    Function that reads in the test sets images as arrays
    parameters:test_set_indices: The indices of the the images in the test set
                 input_dir_test: The directory where the input png images are stored
                output_dir_test: The directory where the output png images are stored
    """
    Test_set_X = np.zeros((len(test_set_indices), 801, 401, 1), dtype=np.float32)
    Test_set_y = np.zeros((len(test_set_indices), 801, 401, 1), dtype=np.float32)
    for i,n in enumerate(test_set_indices):
        img = Image.open(input_dir_test+'/input_%03d.png' % (n+1))
        in_img_input = tf.keras.preprocessing.image.img_to_array(img)
        in_img_input = skimage.transform.resize(in_img_input , (801 , 401 , 1) , mode = 'constant' , preserve_range = True)
        Test_set_X[i] =in_img_input / 255.0

        img = Image.open(output_dir_test+'/target_%03d.png' % (n+1))
        in_img_output = tf.keras.preprocessing.image.img_to_array(img)
        in_img_output = skimage.transform.resize(in_img_output , (801 , 401 , 1) , mode = 'constant' , preserve_range = True)
        Test_set_y[i] =in_img_output / 255.0    
    print('Done')
    return Test_set_X, Test_set_y
#Save numpy files for all the images in the 3 test sets 
for dataset in ['','CIRS_','carotid_']:
    if dataset == '':
        test_set_indices = range(535,669)
        input_dir_test = 'bandLimited'
        output_dir_test = 'groundTruth'
    if dataset == 'CIRS_':
        test_set_indices = range(11)
        input_dir_test = 'CIRSBandLimited'
        output_dir_test = 'CIRSGroundTruth'
    if dataset =='carotid_':
        test_set_indices = range(70)
        input_dir_test = 'carotidBandLimited'
        output_dir_test = 'carotidGroundTruth'
    Test_set_X, Test_set_y = CreateTestSet(test_set_indices,input_dir_test,output_dir_test)
    prediction_images = np.zeros((len(Test_set_X),801,401),dtype=np.float32)
    for i in range(len(Test_set_X)):
        prediction_image = model.predict(np.expand_dims(Test_set_X[i] , 0))
        prediction_image = prediction_image[:,:,:,0]
        prediction_images[i] = prediction_image.squeeze()
    np.save('Model_output_test_'+dataset+'SRCNNmodel1_ultrasound.npy',prediction_images)

Done
Done
Done
